In [1]:
%pip install -U --quiet  langgraph langchain_openai
%pip install -U --quiet tavily-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


# Optional: Configure tracing to visualize and debug the agent
_set_if_undefined("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflexion_1"

_set_if_undefined("OPENAI_API_KEY")




In [4]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama-7b")
# You could also use OpenAI or another provider
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4-turbo-preview")

In [5]:
import json
from typing import Dict, Any
import os
import sys
from APIs.combinedapi import PubMedProcessor

@tool
async def pubmed_paperqa(query: str) -> Dict[str, Any]:
    """ Searches PubmedCentral for papers using a query
    and returns the most relevant chunks using paperQA"""

    max_attempts = 2
    max_results: int = 4
    pubmed_query = query
    doc_query = query
    email = "sanazkazemi@hotmail.com"
    print(f"pubmed_paperqa called with query: {query}, max_results: {max_results}")

    
    pubmed_instance = PubMedProcessor(email)

    # some error handeling in case the API call fails the algorithm will continue.
    results_dict = {}
    for attempt in range(max_attempts):
        try:
            results_dict = await pubmed_instance.full_process(pubmed_query, doc_query, max_results)
            break
        except Exception as e:
            print(f"Error in attempt {attempt+1}: {e}")

    if results_dict is None:
        print("All API call attempts failed. Continuing with empty results.")
        results_dict = {"error": "API calls failed", "results": []}


    # saving the refs to file because the LLM cant do it as it edits in paragraphs - has to be manually done

    file_path = 'RATT_refs.json'

    try:
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            with open(file_path, 'r') as f:
                existing_data = json.load(f)
        else:
            existing_data = []
    except json.JSONDecodeError:
        print("Error reading existing data. Starting with empty list.")

        existing_data = []

    existing_data.append(results_dict)

    with open(file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

                        
    return json.dumps(results_dict, indent=4)

In [6]:
print(pubmed_paperqa.name)
print(pubmed_paperqa.description)
print(pubmed_paperqa.args)

pubmed_paperqa
Searches PubmedCentral for papers using a query
   and returns the most relevant chunks using paperQA
{'query': {'title': 'Query', 'type': 'string'}}


In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers. import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field, ValidationError


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")


class AnswerQuestion(BaseModel):
    """Answer the question. Provide an answer, reflection, and then follow up with search queries to improve the answer."""

    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: list[str] = Field(
        description="1-3 search queries for researching improvements to address the critique of your current answer."
    )


class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    def respond(self, state: list):
        response = []
        for attempt in range(3):
            response = self.runnable.invoke(
                {"messages": state}, {"tags": [f"attempt:{attempt}"]}
            )
            try:
                self.validator.invoke(response)
                return response
            except ValidationError as e:
                state = state + [
                    response,
                    ToolMessage(
                        content=f"{repr(e)}\n\nPay close attention to the function schema.\n\n"
                        + self.validator.schema_json()
                        + " Respond by fixing all validation errors.",
                        tool_call_id=response.tool_calls[0]["id"],
                    ),
                ]
        return response